### Définitions :

- **vu** : consultation d'un produit.
- **atc** (add to cart) : ajout au panier.
- **order** : achat d'un produit.
- **1 parcours** : ensemble des interactions (vu, atc, order) d’un utilisateur durant une seule journée.

---

### Questions :

#### **Question 1** : 
Quel est le délai moyen entre deux achats d'un même utilisateur ?

#### **Question 2** : 
Quel est le "tunnel de déperdition" par `id_produit` ?

- Sans distinction par `id_produit` :
    - Pour 100 "vu", combien d'"add_to_cart" et combien d'"achat" ?
    - Vision par événements et vision globale au niveau de `user_id`.

Exemple :
- 1000 vues → 100 atc (10%) → 10 orders (10%)  
- Taux global : 1% de conversion de "vu" à "order".

- Par `id_produit` :
    - Produit 1 : 1000 vues → 100 atc (10%) → 10 orders (10%).

#### **Question 3** : 
Quel est le taux de conversion pour chaque ASIN (identifiant de produit) ?

- Quel ASIN a le plus de chances d'être acheté lorsqu'il est :
  - Vu ?
  - Ajouté au panier ?

#### **Question 4** : 
Construire l'ensemble des parcours client disponibles, en gardant à l'esprit qu'un parcours doit se faire dans la même journée.

- Exemple 1 :
  - Chemin : `[vu, vu, vu, atc, vu, order]`
  - Résultat : `[1-vu, 2-atc, 3-order]`

- Exemple 2 :
  - Chemin : `[atc, vu, vu, atc, vu, order]`
  - Résultat : `[1-atc, 2-vu, 3-order]`

Format de réponse attendu :
- `[1-vu, 2-atc, 3-order]` : 
  - Combien de fois ?
  - Combien de `user_id` ?
  - Total des ventes (`total_sales`) ?
  - Total des unités vendues (`total_units`) ?
  
- `[1-vu, 2-atc]` : 
  - Combien de fois ?
  - Combien de `user_id` ?

- `[1-atc, 2-order]`
- `[1-vu]`

#### **Question 5** : 
Quels sont les `user_id` ayant acheté 2 produits ou plus dans la même journée ?

#### **Question 6** : 
Avec la seconde table fournie :
- Quels sont les `user_id` ayant acheté le même jour et le même produit ?
- Quels sont les `user_id` qui ont acheté uniquement dans la **table 1** ?
- Quels sont les `user_id` qui ont acheté uniquement dans la **table 2** ?

---

Cette nouvelle organisation rend les questions plus claires et facilite la compréhension des différentes analyses à effectuer.

# <span style="color:green">Data import & cleaning</span>


### <span style="color:green">Import</span>


In [1]:
import pandas as pd
from pandasql import sqldf
import time

df_caudz = pd.read_csv("./data/table_caudz.csv")
df_lowcost = pd.read_csv("./data/table_lowcost.csv")
df_philou = pd.read_csv("./data/table_philou.csv")

df_caudz.head()

,custom_year,custom_month,custom_day,custom_date,custom_user_id,custom_produit_id,custom_event_name,sales,quantity
0,year1,month1,day3,date3,user49,produit1,event_name1,NaN,NaN
1,year1,month1,day5,date5,user849,produit1,event_name1,NaN,NaN
2,year1,month1,day1,date1,user2985,produit1,event_name1,NaN,NaN
3,year1,month1,day2,date2,user4630,produit1,event_name1,NaN,NaN
4,year1,month1,day4,date4,user7535,produit1,event_name1,NaN,NaN


### <span style="color:green">Cleaning</span>


In [2]:
start_time = time.time()
q="""

WITH philou AS (
        SELECT *,
            custom_produit_id || '_philou' AS product,
            'philou' AS marque
        FROM df_philou
    ),
    
    caudz AS (
        SELECT *, 
            custom_produit_id || '_caudz' AS product,
            'caudz' AS marque
        FROM df_caudz
    ),
 
    lowcost AS (
        SELECT *,
            custom_produit_id || '_lowcost' AS product,
            'lowcost' AS marque
        FROM df_lowcost
    ),
    
    union_table AS (
        SELECT *
        FROM philou
            UNION ALL
        SELECT *
        FROM caudz
            UNION ALL
        SELECT *
        FROM lowcost
    )
    
    
SELECT 
    -- On recrée la date au format 'year/month/day'
   CAST(CAST(REPLACE(LOWER(custom_year), 'year', '') AS INT) + 2023 AS VARCHAR) ||'/'|| 
   REPLACE(LOWER(custom_month), 'month', '0') ||'/'|| REPLACE(LOWER(custom_day), 'day', '0') AS date,
   CAST(REPLACE(LOWER(custom_day), 'day', '')AS INT) AS day,
   custom_user_id AS user_id,
   marque,
   CASE
        WHEN custom_event_name = 'event_name1' THEN 'vu'
        WHEN custom_event_name = 'event_name3' THEN 'atc'
        WHEN custom_event_name = 'event_name2' THEN 'order'
    END AS event,
    product,
    sales,
    quantity
    
FROM union_table

"""
df_output = sqldf(q)
end_time = time.time()
execution_time = end_time - start_time
print('execution time =', execution_time)
df_data = df_output
df_output

execution time = 15.473647356033325


,date,day,user_id,marque,event,product,sales,quantity
0,2024/01/01,1,user12504,philou,vu,produit1_philou,NaN,NaN
1,2024/01/01,1,user12504,philou,vu,produit1_philou,NaN,NaN
2,2024/01/05,5,user15299,philou,vu,produit1_philou,NaN,NaN
3,2024/01/01,1,user18045,philou,vu,produit1_philou,NaN,NaN
4,2024/01/01,1,user18045,philou,vu,produit1_philou,NaN,NaN
...,...,...,...,...,...,...,...,...
681168,2024/01/03,3,user59022,lowcost,atc,produit4618_lowcost,NaN,NaN
681169,2024/01/01,1,user78120,lowcost,atc,produit4618_lowcost,NaN,NaN
681170,2024/01/05,5,user98359,lowcost,atc,produit4618_lowcost,NaN,NaN
681171,2024/01/06,6,user98359,lowcost,atc,produit4618_lowcost,NaN,NaN


# <span style="color:green">Questions</span>


### <span style="color:green">Question 1</span>


On veut déterminer le temps moyen entre deux achats pour un même user_id. 

On considère que:
- 1 commande maximum par utilisateur et par jour, ainsi des produits (même de marques différentes), achetés le même jour appartiennent à la même commande
- Les utilisateurs qui n'ont réalisé qu'une seule commande sur notre échantillon sont exclus de notre étude, ainsi seront prius en compte uniquement les utilisateurs avec 2 commandes ou plus

In [3]:
start_time = time.time()
q="""

WITH interval_2_orders AS (
    SELECT 
        *, 
        day - LAG(day) OVER (PARTITION BY user_id ORDER BY day) AS interval_between_orders
    FROM df_data d
    WHERE event = 'order'
        )
        
SELECT AVG(interval_between_orders) AS AVG_per_user_id
FROM interval_2_orders
WHERE interval_between_orders >= 1
"""

df_output = sqldf(q)
end_time = time.time()
execution_time = end_time - start_time
print('execution time =', execution_time)
df_output


execution time = 8.580978870391846


,AVG_per_user_id
0,2.845798


La durée moyenne ente 2 commandes pour un même utilisateur est en moyenne de 2,88 jours.

### <span style="color:green">Question 2</span>


Quel est le "tunnel de déperdition" par `id_produit` ?

- Sans distinction par `id_produit` :
    - Pour 100 "vu", combien d'"add_to_cart" et combien d'"achat" ?
    - Vision par événements et vision globale au niveau de `user_id`.

Exemple :
- 1000 vues → 100 atc (10%) → 10 orders (10%)  
- Taux global : 1% de conversion de "vu" à "order".

- Par `id_produit` :
    - Produit 1 : 1000 vues → 100 atc (10%) → 10 orders (10%).


In [4]:
start_time = time.time()

q="""

WITH 
    --On ne conserve que les produits qui ont été vus plus de 1000 fois
    count_products_vu AS (
        SELECT 
            DISTINCT product,
            COUNT(*) AS count_vu
        FROM df_data
        WHERE event = "vu"
        GROUP BY product
        HAVING COUNT(*)>1000
        ),

    count_products_atc AS (
        SELECT 
            DISTINCT product,
            COUNT(*) AS count_atc
        FROM df_data
        WHERE event = "atc"
        GROUP BY product
        ),

    count_products_order AS (
        SELECT 
            DISTINCT product,
            COUNT(*) AS count_order
        FROM df_data
        WHERE event = "order"
        GROUP BY product
        )
    
SELECT 
    o.product,
    count_vu,
    count_atc,
    count_order,
    (CAST(count_atc AS FLOAT)/CAST(count_vu AS FLOAT)) * 100 AS rate_atc_per_vu,
    (CAST(count_order AS FLOAT)/CAST(count_atc AS FLOAT)) * 100 AS rate_order_per_atc,
    (CAST(count_order AS FLOAT)/CAST(count_vu AS FLOAT)) * 100 AS rate_order_per_vu
FROM count_products_order o
INNER JOIN count_products_atc a ON o.product = a.product
INNER JOIN count_products_vu v ON a.product = v.product
ORDER BY rate_order_per_vu DESC
"""

df_output = sqldf(q)
end_time = time.time()
execution_time = end_time - start_time
print('execution time =', execution_time)
df_question2 = df_output
df_output

execution time = 8.976036787033081


,product,count_vu,count_atc,count_order,rate_atc_per_vu,rate_order_per_atc,rate_order_per_vu
0,produit1500_philou,3241,1287,887,39.709966,68.919969,27.368096
1,produit1466_lowcost,2324,1009,609,43.416523,60.356789,26.204819
2,produit2583_lowcost,1140,359,206,31.491228,57.381616,18.070175
3,produit4456_philou,4170,984,707,23.597122,71.849593,16.954436
4,produit1565_philou,1087,237,146,21.803128,61.603376,13.431463
...,...,...,...,...,...,...,...
78,produit3125_lowcost,2811,157,10,5.585201,6.369427,0.355745
79,produit1982_lowcost,3838,77,11,2.006253,14.285714,0.286608
80,produit4294_lowcost,1513,92,4,6.080635,4.347826,0.264375
81,produit3719_lowcost,9521,110,24,1.155341,21.818182,0.252074


### <span style="color:green">Question 3</span>


Quel est le taux de conversion pour chaque ASIN (identifiant de produit) ?

- Quel ASIN a le plus de chances d'être acheté lorsqu'il est :
  - Vu ?
  - Ajouté au panier ?

In [5]:
#On repart du df de la question 2
df_question2.head()

,product,count_vu,count_atc,count_order,rate_atc_per_vu,rate_order_per_atc,rate_order_per_vu
0,produit1500_philou,3241,1287,887,39.709966,68.919969,27.368096
1,produit1466_lowcost,2324,1009,609,43.416523,60.356789,26.204819
2,produit2583_lowcost,1140,359,206,31.491228,57.381616,18.070175
3,produit4456_philou,4170,984,707,23.597122,71.849593,16.954436
4,produit1565_philou,1087,237,146,21.803128,61.603376,13.431463


In [6]:
q="""
SELECT product,
    MAX(rate_order_per_vu)
FROM df_question2
"""
sqldf(q)

,product,MAX(rate_order_per_vu)
0,produit1500_philou,27.368096


In [7]:
q="""
SELECT product,
    MAX(rate_order_per_atc)
FROM df_question2
"""
sqldf(q)

,product,MAX(rate_order_per_atc)
0,produit4456_philou,71.849593


### <span style="color:green">Question 4</span>


Construire l'ensemble des parcours client disponibles, en gardant à l'esprit qu'un parcours doit se faire dans la même journée.

- Exemple 1 :
  - Chemin : `[vu, vu, vu, atc, vu, order]`
  - Résultat : `[1-vu, 2-atc, 3-order]`

- Exemple 2 :
  - Chemin : `[atc, vu, vu, atc, vu, order]`
  - Résultat : `[1-atc, 2-vu, 3-order]`

Format de réponse attendu :
- `[1-vu, 2-atc, 3-order]` : 
  - Combien de fois ?
  - Combien de `user_id` ?
  - Total des ventes (`total_sales`) ?
  - Total des unités vendues (`total_units`) ?
  
- `[1-vu, 2-atc]` : 
  - Combien de fois ?
  - Combien de `user_id` ?

- `[1-atc, 2-order]`
- `[1-vu]`

In [8]:
#Pour cet exercice il aurait fallu avoir l'heure à laquelle a eu lieu un event car dans notre cas, les données sont mélangées dans une même journée

start_time = time.time()

q="""
WITH 
    event_with_rn AS (
        SELECT *,
            ROW_NUMBER() OVER(PARTITION BY user_id, date, product ORDER BY date) AS rn
        FROM df_data
    ),
    
    --Table avec pour chaque event son ordre d'apparition
    get_events_idx AS (
        SELECT *,
            MIN(rn),
            ROW_NUMBER() OVER(PARTITION BY user_id, date ORDER BY rn) AS event_occurrence
        FROM event_with_rn
        GROUP BY user_id, date, event, product
        ORDER BY date, rn
    ),
    
    --On détermine le parcours d'un utilisateur pour un jour donné
    get_parcours AS (
        SELECT 
            user_id,
            date,
            MAX(CASE WHEN event_occurrence = 1 THEN event ELSE '' END) || ' ' ||
            MAX(CASE WHEN event_occurrence = 2 THEN event ELSE '' END) || ' ' ||
            MAX(CASE WHEN event_occurrence = 3 THEN event ELSE '' END) AS parcours
        FROM get_events_idx
        GROUP BY user_id, date, product
    )

SELECT parcours, 
    COUNT(*)
FROM get_parcours
GROUP BY parcours
"""
df_output = sqldf(q)
end_time = time.time()
execution_time = end_time - start_time
print('execution time =', execution_time)
df_output

execution time = 20.593374729156494


,parcours,COUNT(*)
0,,59360
1,atc,117
2,order,144
3,vu,38520
4,atc,531
5,order,718
6,order atc,66
7,vu,77074
8,vu atc,3195
9,vu order,2383


### <span style="color:green">Question 5</span>


Quels sont les `user_id` ayant acheté 2 produits ou plus dans la même journée ?

In [9]:
q="""
SELECT
    user_id,
    date,
    COUNT(product) AS nbr_product_bought

FROM df_data
WHERE event='order'
GROUP BY user_id, date
HAVING nbr_product_bought>1

"""
sqldf(q)

,user_id,date,nbr_product_bought
0,user10,2024/01/05,2
1,user100002,2024/01/01,2
2,user100088,2024/01/06,2
3,user100112,2024/01/04,2
4,user100124,2024/01/03,2
...,...,...,...
2829,user99905,2024/01/08,2
2830,user99919,2024/01/08,2
2831,user99954,2024/01/08,2
2832,user99973,2024/01/08,2


### <span style="color:green">Question 6</span>


Avec la seconde table fournie :
- Quels sont les `user_id` ayant acheté le même jour et le même produit ?
- Quels sont les `user_id` qui ont acheté uniquement dans la **table 1** ?
- Quels sont les `user_id` qui ont acheté uniquement dans la **table 2** ?


In [29]:
start_time = time.time()

q="""
-- Liste des produits qui ont été achetés par plusieurs user_id le même jour
SELECT 
    product,
    date, 
    COUNT(DISTINCT user_id) AS nbr_user_id
FROM df_data
WHERE event = 'order'
GROUP BY product, date
HAVING COUNT(DISTINCT user_id)>1
"""

df_output = sqldf(q)
end_time = time.time()
execution_time = end_time - start_time
print('execution time =', execution_time)
df_output

execution time = 8.612295627593994


,product,date,nbr_user_id
0,produit1001_philou,2024/01/08,2
1,produit1007_philou,2024/01/02,5
2,produit1007_philou,2024/01/03,2
3,produit1007_philou,2024/01/04,3
4,produit1007_philou,2024/01/05,5
...,...,...,...
4871,produit972_philou,2024/01/03,2
4872,produit972_philou,2024/01/06,2
4873,produit972_philou,2024/01/07,2
4874,produit978_philou,2024/01/04,2


In [30]:
start_time = time.time()

q="""

--Ensemble des user_id qui ont acheté uniquement du lowcost
SELECT user_id FROM df_data WHERE marque = 'lowcost'
EXCEPT
SELECT user_id FROM df_data WHERE marque = 'caudz'
EXCEPT 
SELECT user_id FROM df_data WHERE marque = 'philou'
"""

df_output = sqldf(q)
end_time = time.time()
execution_time = end_time - start_time
print('execution time =', execution_time)
df_output

execution time = 9.869122982025146


,user_id
0,user102892
1,user102893
2,user102894
3,user102895
4,user102896
...,...
7813,user110705
7814,user110706
7815,user110707
7816,user110708
